In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.proportion import proportions_chisquare
from scipy.stats import chisquare

from bs4 import BeautifulSoup
from collections import defaultdict
import requests
%matplotlib inline
import re
from collections import Counter
import string

In [2]:
def drug_link(letter):
    '''
    INPUT: string
    OUTPUT: list of strings

    Take a query and return a list of all of the etsy usernames who have a
    result on the first page of that query result.
    '''

    url = "http://www.druglib.com/drugindex/rating/%s/" % letter
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    search = soup.find_all('a', attrs={'href':re.compile('/ratingsreviews/.*')})
    drug_link = ["http://www.druglib.com"+ a['href'] for a in search]
    return drug_link#[a for a in search]


def scrap_one_page(test_link):
    html = requests.get(test_link).text
    soup = BeautifulSoup(html, 'html.parser')

    ## age_geneder
    age_geneder = soup.find_all('h2')
    age_geneder = [a.contents[0] for a in age_geneder]

    ## rating
    rating =soup.find_all('img',attrs={'hight':"18px"})
    rating = [a['src'] for a in rating]
    rating = [rating[x:x+10] for x in range(0, len(rating),10)]
    rating = [Counter(a)['/img/red_star.gif'] for a in rating ]

    all_contents = [a.contents[0] for a in soup.find_all('td',attrs={'class':"review3"})]
    ## effective
    effective = [a for a in all_contents if 'Effective' in a]

    ## effects
    effects = [a for a in all_contents if 'Effects' in a]

    all_contents_sorted = []
    i = 0
    for x in range(0, len(all_contents),10):
        all_contents_sorted.append([age_geneder[i]]+[rating[i]] + all_contents[x+1:x+10])

    return all_contents_sorted

In [3]:
all_lo_list = []
for l in string.ascii_lowercase:
    oletter_alist =[]
    for each in  drug_link(l):
        temp_list = scrap_one_page(each)
        oletter_alist = oletter_alist + temp_list
    all_lo_list = all_lo_list + oletter_alist


In [6]:
df_ws = pd.DataFrame(all_lo_list)
df_ws.to_csv("web_scrap.csv")

In [7]:
df_ws.head()

,0,1,2,3,4,5,6,7,8,9,10
0,Abilify review by 26 year old female patient,3,Marginally Effective,Severe Side Effects,bipolar disorder,15mg taken daily for the period of 12 weeks,none,"Cymbalta, 90mg/daily",I didn't notice any benefit at all. Supposedl...,A very uncomfortable inner restlessness was t...,I was prescribed Abilify (15mg/daily) to assi...
1,Abilify review by 26 year old female patient,3,Highly Effective,Moderate Side Effects,bipolar,10mg taken 1/day for the period of 8 mos,"bipolar mood disorder, acne, asthma","zoloft, wellbutrin, clonazepam, tretinoin, cl...",I had severe depression with agitation and mi...,"I became drowsy, however, with adequate sleep...",the abilify decreased the need for daily klon...
2,Abilify review by 26 year old female patient,3,Highly Effective,No Side Effects,depression/anxiety,2 mg taken daily for the period of 3 months,anxiety,Lexapro,Within 1 week of taking the cocktail of Abili...,no side effects have been noticed,I take one pill of each 1st thing in the am.....
3,Abilify review by 26 year old female patient,3,Considerably Effective,Extremely Severe Side Effects,depression not resolved with antidepressant d...,started out at 5mg the 10 & last 15mg taken ...,"Add, depression, poss. bipolar type 2, PTSD, ...","vyvance, ativan lisinopril, zantac, viville d...",While on abilify I can honestly say the depre...,but it caused memory loss and again an incid...,I am only taking ativan & getting psychologic...
4,Abilify review by 26 year old female patient,3,Ineffective,Severe Side Effects,bipolar,2mg to start taken once daily for the period...,anxiety,klonopin,None due to the short time taking drug.,Headache first morning at 4AM that was reliev...,Not much to tell. I was just starting treatm...
